# 资源分配问题的建模

考虑三个工作岗位：测试员、Java开发者和架构师。

考虑三个资源：Carlos、Joe和Monika。

## 数据

每种资源对每个工作的能力由下表的匹配分数表示：

![资源分配问题数据图](util/rap_data.png)


**假设**：每个工作只能分配给一个资源，每个资源只能分配到一个工作。

## 问题陈述

确定一个分配方案，确保每个工作都有人完成，每个资源最多分配一个工作，以最大化总匹配分数。

## 决策变量

决策变量 $x_{r,\; j} = 1$ 表示资源r被分配到工作j，否则为0，其中r=1,2,3且j=1,2,3。

## 约束条件

### 工作约束

对于每个工作j=1,2,3，必须恰好分配一个资源（r=1,2,3）。

约束（测试员=1）：$x_{1,\; 1} + x_{2,\; 1} + x_{3,\; 1} = 1$

约束（Java开发者=2）：$x_{1,\; 2} + x_{2,\; 2} + x_{3,\; 2} = 1$

约束（架构师=3）：$x_{1,\; 3} + x_{2,\; 3} + x_{3,\; 3} = 1$

### 资源约束

对于每个资源r=1,2,3，最多只能分配一个工作。

约束（Carlos=1）：$x_{1,\; 1} + x_{1,\; 2} + x_{1,\; 3}  \leq 1$

约束（Joe=2）：$x_{2,\; 1} + x_{2,\; 2} + x_{2,\; 3}  \leq 1$

约束（Monika=3）：$x_{2,\; 1} + x_{2,\; 2} + x_{2,\; 3}  \leq 1$

## 目标函数

目标函数是在满足工作和资源约束的同时，最大化分配的总匹配分数。

$$
Max \; (53x_{1,\; 1} + 80x_{2,\; 1} + 53x_{3,\; 1}) + (27x_{1,\; 2} + 47x_{2,\; 2} + 73x_{3,\; 2})
+ (13x_{1,\; 3} + 67x_{2,\; 3} + 47x_{3,\; 3})
$$




首先，根据需要安装gurobipy

In [ ]:
%pip install gurobipy

In [1]:
# 导入gurobi库
from gurobipy import *

## 数据
列表R包含三个资源的名称：Carlos、Joe和Monika。

列表J包含三个工作岗位的名称：测试员、Java开发者和架构师。

**数学表示法**

$r \in R$ 表示索引为r的资源属于集合（列表）R。

$j \in J$ 表示索引为j的工作属于集合（列表）J。

In [3]:
# 资源和工作集合
R = ['Carlos', 'Joe', 'Monika']
J = ['Tester', 'JavaDeveloper', 'Architect']

以下“multidict”函数描述了每个资源和工作可能组合的匹配分数。

**数学表示法**

令 $ms_{r,\;j}$ 表示资源 $r \in R$ 与工作 $j \in J$ 的匹配分数。

令 $C_{r,\;j}$ 表示将资源 $r \in R$ 分配给工作 $j \in J$ 的成本。

令 $B$ 表示可用预算。

In [4]:
# 匹配分数数据
combinations, ms, C = multidict({
    ('Carlos', 'Tester'): [53, 1],
    ('Carlos', 'JavaDeveloper'): [27, 1],
    ('Carlos', 'Architect'): [13,1],
    ('Joe', 'Tester'): [80, 2],
    ('Joe', 'JavaDeveloper'): [47, 2],
    ('Joe', 'Architect'): [67, 2],
    ('Monika', 'Tester'): [53, 3] ,
    ('Monika', 'JavaDeveloper'): [73, 3],
    ('Monika', 'Architect'): [47, 3]
})

# 可用预算
#B = 6
B=5

下面的函数生成一个空模型对象“m”，并以字符串“RAP”作为模型名称参数。

In [5]:
# 声明并初始化模型
m = Model('RAP')

Set parameter LicenseID to value 2601452


## 决策变量

决策变量 $x_{r,\; j} = 1$ 表示资源r被分配到工作j，否则为0，其中r=1,2,3且j=1,2,3。

“addVars()”方法定义了模型对象“m”的决策变量。

**数学表示法**

令 $x_{r,\; j} = 1$ 表示资源 $r \in R$ 被分配到工作 $j \in J$，否则为零。

令 $g_{j} = 1$ 表示工作 $j \in J$ 不能被填补，否则为零。这个变量是一个缺口变量，表明某工作无法填补。


In [6]:
# 为RAP模型创建决策变量
#x = m.addVars(combinations, name="assign")
x = m.addVars(combinations, vtype=GRB.BINARY, name="assign")

# 为RAP模型创建缺口变量
g = m.addVars(J, name="gap")

## 工作约束

对于每个工作j=1,2,3，必须恰好分配一个资源（r=1,2,3）。

约束（测试员=1）：$x_{1,\; 1} + x_{2,\; 1} + x_{3,\; 1} + g_{1} = 1$

约束（Java开发者=2）：$x_{1,\; 2} + x_{2,\; 2} + x_{3,\; 2} + g_{2} = 1$

约束（架构师=3）：$x_{1,\; 3} + x_{2,\; 3} + x_{3,\; 3} + g_{3} = 1$

“addConstrs()”方法定义了模型对象“m”的约束条件。

**数学表示法**

对于每个工作 $j \in J$，必须恰好分配一个资源：

$$
\sum_{r \: \in \: R} x_{r,\; j} + g_{j} = 1 
$$



In [7]:
# 创建工作约束
jobs = m.addConstrs((x.sum('*',j) + g[j]  == 1 for j in J), 'job')

## 资源约束

对于每个资源r=1,2,3，最多只能分配一个工作。

约束（Carlos=1）：$x_{1,\; 1} + x_{1,\; 2} + x_{1,\; 3}  \leq 1$

约束（Joe=2）：$x_{2,\; 1} + x_{2,\; 2} + x_{2,\; 3}  \leq 1$

约束（Monika=3）：$x_{2,\; 1} + x_{2,\; 2} + x_{2,\; 3}  \leq 1$

“addConstrs()”方法定义了模型对象“m”的约束条件。

**数学表示法**

对于每个资源 $r \in R$，最多只能分配一个工作：

$$
\sum_{j \: \in \: J} x_{r,\; j} \leq 1 
$$

In [8]:
# 创建资源约束
resources = m.addConstrs((x.sum(r,'*') <= 1 for r in R), 'resource')

## 预算约束

分配资源到工作的总成本应小于或等于可用预算。

$$
\sum_{r \; \in \; R} \sum_{j \; \in \; J} C_{r, j}x_{r, j} \leq B
$$

In [9]:
budget = m.addConstr((x.prod(C) <= B), 'budget')

## 目标函数

目标函数是最大化分配的总匹配分数。

$$
Max \; (53x_{1,\; 1} + 80x_{2,\; 1} + 53x_{3,\; 1}) + (27x_{1,\; 2} + 47x_{2,\; 2} + 73x_{3,\; 2})
+ (13x_{1,\; 3} + 67x_{2,\; 3} + 47x_{3,\; 3})
$$

“setObjective()”方法定义了模型对象“m”的目标函数。

**数学表示法**

注意，
$$
(53x_{1,\; 1} + 80x_{2,\; 1} + 53x_{3,\; 1}) = \sum_{r \; \in \; R} ms_{r,1}x_{r,1} \\
(27x_{1,\; 2} + 47x_{2,\; 2} + 73x_{3,\; 2}) = \sum_{r \; \in \; R} ms_{r,2}x_{r,2} \\
(13x_{1,\; 3} + 67x_{2,\; 3} + 47x_{3,\; 3})  = \sum_{r \; \in \; R} ms_{r,3}x_{r,3}
$$

因此，目标函数可以表示为：

$$
Max \; \sum_{j \; \in \; J} \sum_{r \; \in \; R} ms_{r,j}x_{r,j} -BigM \sum_{j \in J} g_{j}
$$


In [10]:
# 未填补工作岗位的惩罚
BIGM =101

# 目标是最大化分配的总匹配分数
m.setObjective(x.prod(ms) -BIGM*g.sum(), GRB.MAXIMIZE)

In [ ]:
# 保存模型以供检查
m.write('RAP3.lp')

In [11]:
# 运行优化引擎
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) Ultra 5 125H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 18 logical processors, using up to 18 threads

Optimize a model with 7 rows, 12 columns and 30 nonzeros
Model fingerprint: 0xa1231a12
Variable types: 3 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve time: 0.01s
Presolved: 7 rows, 12 columns, 30 nonzeros
Variable types: 0 continuous, 12 integer (12 binary)
Found heuristic solution: objective 52.0000000

Root relaxation: objective 1.350000e+02, 4 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  135.00000    0    2   52.00000  135.00000   160

In [12]:
# 显示决策变量的最优值
for v in m.getVars():
	if (abs(v.x) > 1e-6):
		print(v.varName, v.x)

# 显示最优目标函数值
print('Optimal objective function value', m.objVal)   

# 根据分配变量计算总匹配分数
total_matching_score = 0
for [r, j] in combinations:
    if (abs(x[r, j].x) > 1e-6):
        total_matching_score = total_matching_score + ms[r, j]*x[r, j].x

print('Total matching score: ', total_matching_score)  


assign[Joe,Tester] 1.0
assign[Monika,JavaDeveloper] 1.0
gap[Architect] 1.0
Optimal objective function value 52.0
Total matching score:  153.0
